# 金融风控页面案例

## 【实现】风控业务案例

### 案例背景介绍

- 通过对业务数据分析了解信贷业务状况
- 数据集说明
  - 从开源数据改造而来，基本反映真实业务数据
  - 销售，客服可以忽略
  - 账单周期，放款日期
  - 账单金额-实收金额 = 未收金额
  - 应付日期为还款时间
  - 账期分成两种：60天和90天
  - 实际到账日为空白，说明没还钱

In [ ]:
import pandas as pd
from pyecharts.charts import *
from pyecharts import options as opts

df1 = pd.read_excel('./data/业务数据.xls') 

# 要使用原始数据构建新指标，所以保留原始数据，copy新的数据，在新的数据中创建新指标
df2 = df1.copy()

# head() 输出前五条数据
df2.head()

In [ ]:
df2.info()

In [ ]:
df2.describe()

In [ ]:
# 获取最大的日期，作为当前时间
today_time = pd.to_datetime(df2.实际到账日.fillna('0').max())

#给缺失值填充0
df2['实收金额'] = df2.实收金额.fillna(0)
df2['开票金额'] = df2.开票金额.fillna(0)
df2['未收金额'] = df2.未收金额.fillna(0)

#把时间类型转换为datetime类型
df2['账单周期'] = pd.to_datetime(df2.账单周期)
df2['应付日期'] = pd.to_datetime(df2.应付日期)

df2['实际到账日'] = pd.to_datetime(df2.实际到账日).fillna(today_time)

In [ ]:
df2['是否到期'] = df2.apply(lambda x : 0 if x.应付日期 > today_time else 1,axis=1)

#map可以看做是apply，效果类似
df2['是否到期90天'] =  ( today_time - df2.应付日期 ).map(lambda x : 1 if x.days >= 90 else 0)

df2['未收金额2'] =  (df2.账单金额 - df2.实收金额)

df2['历史逾期天数'] = df2.apply(lambda x : (x.实际到账日 -  x.应付日期).days if x.未收金额2 == 0  else  (today_time - x.应付日期).days,axis=1)

#df2['当前逾期天数'] = df2.apply(lambda x : (x.历史逾期天数) if x.未收金额2 > 0  else 0 ,axis = 1)
df2['当前逾期天数'] = df2.apply(lambda x:(today_time - x['应付日期']).days if x['未收金额2'] > 0 else 0,axis=1)

df2

In [ ]:
df3 =df2.copy()
#创建’账单季度‘字段，将日期转换成季度，to_period函数可以转换为季度信息
df3['账单季度'] = df3['账单周期'].map(lambda x : x.to_period('Q'))
#提取2017年3季度到2018年4季度数据
df3 = df3[(df3['账单季度']<='2018Q4') & (df3['账单季度']>='2017Q3')]
df3.shape

df3


In [ ]:
#账单金额
fn1 = df3.groupby('账单季度')[['账单金额']].sum()
fn1.columns = ['账单金额']
fn1

In [ ]:
#90天到期金额
df4 = df3[(df3.是否到期90天 == 1)]

fn2 = df4.groupby('账单季度')[['账单金额']].sum()
fn2.columns = ['到期金额']
fn2

In [ ]:
#当前逾期90+金额
df4 = df3[(df3.是否到期90天 == 1)]
fn3 = df4.groupby('账单季度')[['未收金额2']].sum()
fn3.columns = ['当前逾期90+金额']
fn3

In [ ]:
dfs = [fn1,fn2,fn3]
final1 = pd.concat(dfs,axis=1)
final1

In [ ]:
final1['90+净坏账率'] = round(final1['当前逾期90+金额'] / final1.到期金额,3)
final1

In [ ]:
bar = (
    Bar()
    .add_xaxis(list(final1.index.values.astype(str)))
    .add_yaxis(
        "账单金额",
        list(final1.账单金额),
        yaxis_index=0,
        color="#5793f3",
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="90+净坏账率"),
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="90+净坏账率",
            type_="value",
            min_=0,
            max_=0.014,
            position="right",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="#d14a61")
            ),
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
        )
    )
)
line = (
    Line()
    .add_xaxis(list(final1.index.values.astype(str)))
    .add_yaxis(
        "90+净坏账率",
        list(final1['90+净坏账率']),
        yaxis_index=1,
        color="#675bba",
        label_opts=opts.LabelOpts(is_show=False),
    )
)
# 在 ./chart 目录下生成 90+净坏账率.html
bar.overlap(line).render('./chart/90+净坏账率.html')